In [1]:
%load_ext autoreload
%autoreload 2

# Test connection

In [2]:
# setup logger
from loguru import logger

In [3]:
import psycopg2
conn = psycopg2.connect(
    host='default.561130499334.eu-central-1.redshift-serverless.amazonaws.com',
    port="5439",
    user="awsuser",
    password="R3dsh1ft",
    database='dev'
)
cursor = conn.cursor()
conn.autocommit = True  # do not need commit then

In [4]:
from pprint import pformat

def run(query: str):
    q = query.strip()
    assert q.endswith(";")
    logger.info(f"Running\n {q}")
    cursor.execute(q)
    try:
        result = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        rows = [dict(zip(column_names, row)) for row in result]
        logger.success(f"{pformat(rows)}")
    except psycopg2.ProgrammingError as e:
        if "no results to fetch" in str(e):
            logger.success("No results to fetch")
        else:
            logger.exception(e)
            conn.rollback()
            raise e
    # conn.commit() # not necessary with autocommit


run('SELECT * FROM DummyTable;')

2023-08-04 00:27:49.686 | INFO     | __main__:run:6 - Running
 SELECT * FROM DummyTable;
2023-08-04 00:27:49.713 | SUCCESS  | __main__:run:12 - [{'age': 30, 'id': 1, 'name': 'John'},
 {'age': 25, 'id': 2, 'name': 'Jane'},
 {'age': 40, 'id': 3, 'name': 'Mike'}]


In [ ]:
# if necessary: close connection
cursor.close()
conn.close()

# Try to diagnose errors and events

In [18]:
# s="""SELECT * FROM SVL_QLOG LIMIT 3;"""
# -- grant select on svv_redshift_databases to awsuser;
# -- grant select on svv_all_schemas to awsuser;
# -- grant select on svv_all_tables to awsuser;
# grant select on stv_recents to awsuser

# s="GRANT ALL PRIVILEGES on all tables in schema public to awsuser;"
s="SELECT * FROM SVL_QLOG LIMIT 3;"
run(s)

2023-08-04 00:33:30.647 | INFO     | __main__:run:6 - Running
 SELECT * FROM SVL_QLOG LIMIT 3;


InsufficientPrivilege: permission denied for relation svl_qlog


# Using my original SQL statements

## Create tables

In [8]:
from old import sql_queries

In [25]:
for q in sql_queries.drop_table_queries:
    run(q)

2023-07-24 20:04:44.216 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS event_stage_table;
2023-07-24 20:04:44.272 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 20:04:45.409 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS song_stage_table;
2023-07-24 20:04:45.477 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 20:04:46.745 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS songplays;
2023-07-24 20:04:46.796 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 20:04:46.832 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS songs;
2023-07-24 20:04:46.879 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 20:04:46.909 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS users;
2023-07-24 20:04:46.961 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 20:04:46.993 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS artists;
2023-07-24 20:04:47.058 | SUCCESS  | __main__:run:

In [15]:
for q in sql_queries.create_table_queries:
    run(q)

2023-07-24 19:34:57.312 | INFO     | __main__:run:5 - Running
 CREATE TABLE event_stage_table (
    artist VARCHAR(255),
    auth VARCHAR(255),
    firstName VARCHAR(255),
    gender CHAR(1),
    itemInSession SMALLINT,
    lastName VARCHAR(255),
    length FLOAT,
    level VARCHAR(50),
    location VARCHAR(255),
    method VARCHAR(10),
    page VARCHAR(50),
    registration BIGINT,
    sessionId INTEGER,
    song VARCHAR(255),
    status SMALLINT,
    ts BIGINT,
    userAgent VARCHAR(255),
    userId INTEGER
);
2023-07-24 19:34:57.375 | SUCCESS  | __main__:run:12 - No results to fetch
2023-07-24 19:34:58.497 | INFO     | __main__:run:5 - Running
 CREATE TABLE song_stage_table (
    artist_id VARCHAR(50),
    artist_latitude FLOAT,
    artist_location VARCHAR(500),
    artist_longitude FLOAT,
    artist_name VARCHAR(500),
    duration FLOAT,
    num_songs INT,
    song_id VARCHAR(50),
    title VARCHAR(500),
    year INT
);
2023-07-24 19:34:58.578 | SUCCESS  | __main__:run:12 - No resu

## Load tables into staging

In [16]:
for q in sql_queries.copy_table_queries:
    run(q)

2023-07-24 19:35:36.050 | INFO     | __main__:run:5 - Running
 COPY event_stage_table
FROM 's3://udacity-dataengineer-pipeline-project-s3/log_data'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
--JSON 'auto' -- doesn't work, because keys don't match perfectly
JSON 's3://udacity-dataengineer-pipeline-project-s3/log_json_path.json'
STATUPDATE ON
MAXERROR 1
COMPUPDATE OFF;
2023-07-24 19:35:39.232 | SUCCESS  | __main__:run:12 - No results to fetch
2023-07-24 19:35:40.173 | INFO     | __main__:run:5 - Running
 COPY song_stage_table
FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/A/A/A/' -- use subset
-- FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
JSON 'auto'
STATUPDATE ON
MAXERROR 1
COMPUPDATE OFF;
2023-07-24 19:35:56.463 | SUCCESS  | __main__:run:12 - No results to fetch


In [19]:
import utils
for t in ["event_stage_table", "song_stage_table"]:
    run(utils.get_list_query(t))

2023-07-24 19:42:04.985 | INFO     | __main__:run:5 - Running
 SELECT * FROM event_stage_table LIMIT 3;
2023-07-24 19:42:05.044 | SUCCESS  | __main__:run:11 - [{'artist': 'Dee Dee Bridgewater',
  'auth': 'Logged In',
  'firstname': 'Lily',
  'gender': 'F',
  'iteminsession': 38,
  'lastname': 'Koch',
  'length': 318.64118,
  'level': 'paid',
  'location': 'Chicago-Naperville-Elgin, IL-IN-WI',
  'method': 'PUT',
  'page': 'NextSong',
  'registration': 1541048010796,
  'sessionid': 818,
  'song': 'La Vie En Rose',
  'status': 200,
  'ts': 1542845032796,
  'useragent': '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 '
               'Safari/537.36"',
  'userid': 15},
 {'artist': "Tim O'brien",
  'auth': 'Logged In',
  'firstname': 'Lily',
  'gender': 'F',
  'iteminsession': 39,
  'lastname': 'Koch',
  'length': 176.14322,
  'level': 'paid',
  'location': 'Chicago-Naperville-Elgin, IL-IN-WI',
  'm

## Insert data

In [20]:
for q in sql_queries.insert_table_queries:
    run(q)

2023-07-24 19:44:54.587 | INFO     | __main__:run:5 - Running
 INSERT INTO users (user_id, first_name, last_name, gender, level)
SELECT DISTINCT userId, firstName, lastName, gender, level
FROM event_stage_table
WHERE userId IS NOT NULL;
2023-07-24 19:45:03.167 | SUCCESS  | __main__:run:14 - No results to fetch
2023-07-24 19:45:04.444 | INFO     | __main__:run:5 - Running
 INSERT INTO artists (artist_id, name, location, latitude, longitude)
SELECT DISTINCT artist_id, artist_name, artist_location, artist_latitude, artist_longitude
FROM song_stage_table;
2023-07-24 19:45:13.002 | SUCCESS  | __main__:run:14 - No results to fetch
2023-07-24 19:45:14.131 | INFO     | __main__:run:5 - Running
 INSERT INTO time (start_time, hour, day, week, month, year, weekday)
SELECT DISTINCT
    TIMESTAMP 'epoch' + ts/1000 * INTERVAL '1 second' AS start_time,
    EXTRACT(hour FROM start_time) AS hour,
    EXTRACT(day FROM start_time) AS day,
    EXTRACT(week FROM start_time) AS week,
    EXTRACT(month FROM 

In [24]:
for t in ["songplays","users","songs","artists","time"]:
    # run(f"DELETE FROM {t};")
    run(utils.get_drop_query(t))

2023-07-24 19:58:29.165 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS songplays;
2023-07-24 19:58:29.227 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 19:58:30.315 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS users;
2023-07-24 19:58:30.386 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 19:58:31.323 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS songs;
2023-07-24 19:58:31.380 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 19:58:33.451 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS artists;
2023-07-24 19:58:33.523 | SUCCESS  | __main__:run:16 - No results to fetch
2023-07-24 19:58:34.524 | INFO     | __main__:run:7 - Running
 DROP TABLE IF EXISTS time;
2023-07-24 19:58:34.575 | SUCCESS  | __main__:run:16 - No results to fetch


# Using new statements

## Create tables

In [20]:
from old import utils

for t in ["artists","songplays", "songs","staging_events","staging_songs","time","users"]:
    run(utils.get_drop_query(t))

2023-08-04 00:44:30.661 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS artists;
2023-08-04 00:44:30.695 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:30.696 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS songplays;
2023-08-04 00:44:30.728 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:30.730 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS songs;
2023-08-04 00:44:30.765 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:30.766 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS staging_events;
2023-08-04 00:44:30.798 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:30.799 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS staging_songs;
2023-08-04 00:44:31.783 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:31.786 | INFO     | __main__:run:6 - Running
 DROP TABLE IF EXISTS time;
2023-08-04 00:44:32.670 | SUCCESS  | __main__:run:15 - No

In [21]:
from pathlib import Path

sql_filepath = Path().resolve().parents[1] / "template/create_tables.sql"
sql_filepath.exists()

True

In [22]:
txt = sql_filepath.read_text()
create_queries = [s.replace("\n","").replace("\t","") for s in txt.split(";")]
create_queries

['CREATE TABLE public.artists (artistid varchar(256) NOT NULL,name varchar(256),location varchar(256),lattitude numeric(18,0),longitude numeric(18,0))',
 'CREATE TABLE public.songplays (playid varchar(32) NOT NULL,start_time timestamp NOT NULL,userid int4 NOT NULL,"level" varchar(256),songid varchar(256),artistid varchar(256),sessionid int4,location varchar(256),user_agent varchar(256),CONSTRAINT songplays_pkey PRIMARY KEY (playid))',
 'CREATE TABLE public.songs (songid varchar(256) NOT NULL,title varchar(256),artistid varchar(256),"year" int4,duration numeric(18,0),CONSTRAINT songs_pkey PRIMARY KEY (songid))',
 'CREATE TABLE public.staging_events (artist varchar(256),auth varchar(256),firstname varchar(256),gender varchar(256),iteminsession int4,lastname varchar(256),length numeric(18,0),"level" varchar(256),location varchar(256),"method" varchar(256),page varchar(256),registration numeric(18,0),sessionid int4,song varchar(256),status int4,ts int8,useragent varchar(256),userid int4)',

In [23]:
for q in create_queries:
    if q:
        run(q + ";")

2023-08-04 00:44:45.037 | INFO     | __main__:run:6 - Running
 CREATE TABLE public.artists (artistid varchar(256) NOT NULL,name varchar(256),location varchar(256),lattitude numeric(18,0),longitude numeric(18,0));
2023-08-04 00:44:46.329 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:46.332 | INFO     | __main__:run:6 - Running
 CREATE TABLE public.songplays (playid varchar(32) NOT NULL,start_time timestamp NOT NULL,userid int4 NOT NULL,"level" varchar(256),songid varchar(256),artistid varchar(256),sessionid int4,location varchar(256),user_agent varchar(256),CONSTRAINT songplays_pkey PRIMARY KEY (playid));
2023-08-04 00:44:47.519 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-04 00:44:47.522 | INFO     | __main__:run:6 - Running
 CREATE TABLE public.songs (songid varchar(256) NOT NULL,title varchar(256),artistid varchar(256),"year" int4,duration numeric(18,0),CONSTRAINT songs_pkey PRIMARY KEY (songid));
2023-08-04 00:44:48.476 | SUCCESS  | __main__:run:1

## Copy tables

In [24]:
q2="""
COPY staging_songs
FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/A/A/A/' -- use subset
-- FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
JSON 'auto'
STATUPDATE OFF
MAXERROR 1
COMPUPDATE OFF;
"""
run(q2)

2023-08-04 00:44:58.357 | INFO     | __main__:run:6 - Running
 COPY staging_songs
FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/A/A/A/' -- use subset
-- FROM 's3://udacity-dataengineer-pipeline-project-s3/song_data/'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
JSON 'auto'
STATUPDATE OFF
MAXERROR 1
COMPUPDATE OFF;


InternalError_: abort query
DETAIL:  
  -----------------------------------------------
  error:  abort query
  code:      1020
  context:   system requested abort
  query:     2418347[child_sequence:1]
  location:  queryabort.hpp:410
  process:   padbmaster [pid=1073905947]
  -----------------------------------------------



In [ ]:
run(utils.get_list_query("staging_songs"))

In [14]:
q1 ="""
COPY staging_events
FROM 's3://udacity-dataengineer-pipeline-project-s3/log_data'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
JSON 's3://udacity-dataengineer-pipeline-project-s3/log_json_path.json'
STATUPDATE ON
MAXERROR 1
COMPUPDATE OFF;
"""
run(q1)

2023-08-02 19:11:25.444 | INFO     | __main__:run:6 - Running
 COPY staging_events
FROM 's3://udacity-dataengineer-pipeline-project-s3/log_data'
IAM_ROLE 'arn:aws:iam::561130499334:role/my-redshift-service-role'
JSON 's3://udacity-dataengineer-pipeline-project-s3/log_json_path.json'
STATUPDATE ON
MAXERROR 1
COMPUPDATE OFF;
2023-08-02 19:11:28.459 | SUCCESS  | __main__:run:15 - No results to fetch


In [15]:
run(utils.get_list_query("staging_events"))

2023-08-02 19:11:28.529 | INFO     | __main__:run:6 - Running
 SELECT * FROM staging_events LIMIT 3;
2023-08-02 19:11:31.467 | SUCCESS  | __main__:run:12 - [{'artist': None,
  'auth': 'Logged In',
  'firstname': 'Walter',
  'gender': 'M',
  'iteminsession': 0,
  'lastname': 'Frye',
  'length': None,
  'level': 'free',
  'location': 'San Francisco-Oakland-Hayward, CA',
  'method': 'GET',
  'page': 'Home',
  'registration': Decimal('1540919166796'),
  'sessionid': 38,
  'song': None,
  'status': 200,
  'ts': 1541105830796,
  'useragent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) '
               'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 '
               'Safari/537.36"',
  'userid': 39},
 {'artist': None,
  'auth': 'Logged In',
  'firstname': 'Kaylee',
  'gender': 'F',
  'iteminsession': 0,
  'lastname': 'Summers',
  'length': None,
  'level': 'free',
  'location': 'Phoenix-Mesa-Scottsdale, AZ',
  'method': 'GET',
  'page': 'Home',
  'registration': Decimal('154

## Insert

In [1]:
from ...template.plugins.helpers.sql_queries import SqlQueries

dct = {k: v for k,v in vars(SqlQueries).items() if k.endswith("_insert")}
dct

{'songplay_table_insert': '\n        INSERT INTO public.songplays (playid, start_time, userid, "level", songid, artistid, sessionid, location, user_agent)\n        SELECT\n                md5(events.sessionid || events.start_time) songplay_id,\n                events.start_time, \n                events.userid, \n                events.level, \n                songs.song_id, \n                songs.artist_id, \n                events.sessionid, \n                events.location, \n                events.useragent\n                FROM (SELECT TIMESTAMP \'epoch\' + ts/1000 * interval \'1 second\' AS start_time, *\n            FROM staging_events\n            WHERE page=\'NextSong\') events\n            LEFT JOIN staging_songs songs\n            ON events.song = songs.title\n                AND events.artist = songs.artist_name\n                AND events.length = songs.duration\n    ',
 'user_table_insert': '\n        INSERT INTO public.users (userid, first_name, last_name, gender, "lev

In [17]:
for k,v in dct.items():
    run(v + ";")

2023-08-02 19:11:40.290 | INFO     | __main__:run:6 - Running
 INSERT INTO public.songplays (playid, start_time, userid, "level", songid, artistid, sessionid, location, user_agent)
        SELECT
                md5(events.sessionid || events.start_time) songplay_id,
                events.start_time, 
                events.userid, 
                events.level, 
                songs.song_id, 
                songs.artist_id, 
                events.sessionid, 
                events.location, 
                events.useragent
                FROM (SELECT TIMESTAMP 'epoch' + ts/1000 * interval '1 second' AS start_time, *
            FROM staging_events
            WHERE page='NextSong') events
            LEFT JOIN staging_songs songs
            ON events.song = songs.title
                AND events.artist = songs.artist_name
                AND events.length = songs.duration
    ;
2023-08-02 19:11:48.427 | SUCCESS  | __main__:run:15 - No results to fetch
2023-08-02 19:11:48.428 | 

In [18]:
for t in ["public.songs", "public.users"]:
    run(utils.get_list_query(t))

2023-08-02 19:12:03.245 | INFO     | __main__:run:6 - Running
 SELECT * FROM public.songs LIMIT 3;
2023-08-02 19:12:07.475 | SUCCESS  | __main__:run:12 - [{'artistid': 'AR1KTV21187B9ACD72',
  'duration': Decimal('343'),
  'songid': 'SOSMJFC12A8C13DE0C',
  'title': 'Is That All There Is?',
  'year': 0},
 {'artistid': 'ARA23XO1187B9AF18F',
  'duration': Decimal('192'),
  'songid': 'SOKTJDS12AF72A25E5',
  'title': 'Drown In My Own Tears (24-Bit Digitally Remastered 04)',
  'year': 0},
 {'artistid': 'ARSVTNL1187B992A91',
  'duration': Decimal('129'),
  'songid': 'SOEKAZG12AB018837E',
  'title': "I'll Slap Your Face (Entertainment USA Theme)",
  'year': 2001}]
2023-08-02 19:12:07.478 | INFO     | __main__:run:6 - Running
 SELECT * FROM public.users LIMIT 3;
2023-08-02 19:12:07.566 | SUCCESS  | __main__:run:12 - [{'first_name': 'Stefany',
  'gender': 'F',
  'last_name': 'White',
  'level': 'free',
  'userid': 83},
 {'first_name': 'Brantley',
  'gender': 'M',
  'last_name': 'West',
  'level':